# **Análisis y Visualización de Datos**

## **Actividades**

#### **Análisis descriptivo**

* Transformar cada campo de acuerdo con su tipo de variable (entero, fecha, string, etc).

* Visualizar la distribución de datos de cada campo utilizando gráficos que consideren adecuados. 

#### **Análisis estadístico**

* Caracterizar a los pacientes de la institución en términos de edad, sexo y prestador. Para el caso de la edad, determinar si los datos siguen alguna alguna distribución conocida. Para el sexo, determinar si hay una diferencia significativa entre hombres y mujeres.

* Determinar la cantidad de turnos asignados por pacientes, y la cantidad de ausencias.

* **Observación:** El campo "Paciente" corresponde al nombre y apellido del paciente, por lo que no contamos con una identificación unívoca de cada persona, dado que los nombres se pueden repetir. ¿Es esto realmente un problema? ¿Se puede resolver?

* Elegir (al menos) tres variables, las cuales crean que pueden estar correlacionadas con la tasa de ausentismo. Para cada una de ellas calcular la probabilidad de que un turno esté ausente, condicionada a esa variable. 

* En el ámbito de la medicina es común ver que se utilizan descriptores estadísticos particulares. Uno de ellos es el Odds Ratio, el cual mide las chances de que los valores de dos variables sean observados en conjunto [1]. Para cada variable, calcular los Odds Ratio (OR) con respecto a la variable objetivo (austentismo), y determinar su intervalo de confianza utilizando el método que consideren adecuado.

* Comparar los resultdos de los dos puntos anteriores. Si existen diferencias, discutir cuáles son y a qué se deben.

#### **Cantidad de turnos en el tiempo:**

* ¿Cuáles son los horarios más demandados? ¿Existe una hora pico?

* ¿Qué días de la semana se dan más turnos? ¿Qué días presentan mayor ausentismo?

* ¿Cómo evoluciona la cantidad de turnos y la cantidad de ausencias a lo largo del año? ¿Existe alguna tendencia?

#### **Opcional**

* Realizar un gráfico Ranking-Frecuencia [2] para la cantidad de turnos por paciente. ¿Se cumple la ley de Zipf? Realizar el mismo análisis, pero separando turnos atendidos y turnos ausentes.

## **Entrega**

**Formato de entrega:** Deberán entregar el análisis realizado, junto con los códigos con los que hicieron el análisis. Pueden optar por un único Notebook que alterne celdas de código con una descripción detallada, o bien, un Notebook más centrado en los códigos, junto con una presentación que detalle el análisis.

**Fecha de entrega:** 22/6

## **Referencias:**

* [1] Explicación de Odds Ratio: https://www.youtube.com/watch?v=8nm0G-1uJzA
* [2] Rank-frequency plot: https://en.wikipedia.org/wiki/Rank-size_distribution#:~:text=Rank%2Dsize%20distribution%20is%20the,in%20decreasing%20order%20of%20size.&text=This%20is%20also%20known%20as,city%20size%20or%20word%20frequency.






Importamos librerías y configuración general

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.options.display.max_columns = 30
pd.options.display.max_rows = 50

Leemos el dataset

In [2]:
data_dir = os.path.join('..', 'data')
data_file_name = 'dataset_turnos_2019.csv'
full_data_file_name = os.path.join(data_dir, data_file_name)
df = pd.read_csv(full_data_file_name, sep=';')
print(df.shape)
df.tail()

(1194014, 13)


,CentroDeAtencion,Prestador,Recurso,Servicio,FechaDelTurno,EstadoDelTurno,Paciente,Sexo,Edad,PrestacionAsignada,NombreDelCanal,EsSobreTurno,TipoDeTurnoAsignado
1194009,2,-,fa089caaca143ad1ff284640fb4cd2d7,RESONANCIA,2019-12-30 23:30:00,Libre,NaN,,0,NaN,NaN,NO,NaN
1194010,1,OSDE,cba3f4d0f0775d3211c12cbca0ab3781,ECOGRAFIA,2019-12-31 09:24:00,Atendido,15a7d517fa140ac76f9a7a79de343d81,M,44,ECOGRAFIA COMPLETA DE ABDOMEN,NaN,SI,Práctica
1194011,1,OMINT S. A. DE SERVICIOS,cba3f4d0f0775d3211c12cbca0ab3781,ECOGRAFIA,2019-12-31 09:27:00,Atendido,2bd3a6fd81b29758aa12a13d210e6fdc,M,53,ECOGRAFIA DE PARTES BLANDAS,NaN,SI,Práctica
1194012,1,SANCOR,cba3f4d0f0775d3211c12cbca0ab3781,ECOGRAFIA,2019-12-31 11:29:00,Atendido,d72ffdcc0121edd182da6a32578e4bb1,F,28,ECOGRAFIA TRANSVAGINAL,NaN,SI,Práctica
1194013,1,MEDIFE ASOCIACIÓN CIVIL,a2073c1d145a1b5fd133f927fbbd7913,RADIOGRAFIA,2019-12-31 14:23:00,Atendido,71f43803027051f252e553746b44820b,M,2,Prestaciones multiples,NaN,SI,Práctica


Analizamos los tipos de datos de cada campo inferidos por `pandas`

In [3]:
dtypes = pd.DataFrame(df.dtypes)
dtypes.index.name = 'Campo'
dtypes.rename(columns={0: 'Tipo'})

,Tipo
Campo,
CentroDeAtencion,int64
Prestador,object
Recurso,object
Servicio,object
FechaDelTurno,object
EstadoDelTurno,object
Paciente,object
Sexo,object
Edad,int64


Transformamos el campo **FechaDelTurno** en *datetime*

In [4]:
df.FechaDelTurno = pd.to_datetime(df.FechaDelTurno)

Buscamos posibles duplicados

In [5]:
df.duplicated().sum()

0

Verificamos que los turnos estén ordenados por fecha

In [6]:
## Este valor debería ser no negativo
df.sort_values(by='FechaDelTurno').FechaDelTurno.diff().min()

Timedelta('0 days 00:00:00')